# Web Scrapping lab

In this lab you will scrappe this [website](https://books.toscrape.com/) of books.

You have to create a Pandas DataFrame with all the books listed in the page. Each row of the DataFrame should contain information of each book. In particular, the DataFrmae must contain:

* category
* title
* price
* stock availability
* star rating (number of stars)
* description
* UPC

Happy scrapping!



# Server verification

Load the needed libraries, and make sure thar you can obtain the correct status code.

In [23]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "http://books.toscrape.com/"
response = requests.get(url)


# Book categories

Create the code to collect the **relative urls** from the left panel to obtain a list with all the book categories.

In [24]:
soup = BeautifulSoup(response.content, "html.parser")
categories = soup.select('.nav-list a')

cat_urls = []
cat_names = []
for category in categories:
    cat_urls.append(category["href"])
    cat_names.append(category.text.strip())

# Books in a given category

Use. web scrapping and list comprehension to obtain the **absolute** url of each book to be scraped

In [25]:
def get_book_urls(category_url):
    response = requests.get(url + category_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    book_links = soup.select('h3 a')
    book_urls = [url + 'catalogue/' + link.get('href') for link in book_links]
    return book_urls


# Book details

Create a Python function that given a book_url as an input returns a dictionary with the following structure:

```Python
{"Title": title, "Price": price, "Availability": availability, "Rating": rating, "Description": description, "UPC": upc}
```

where `description` should contain the book's summary given in the Product description, and the values are the book's associated information.

In [26]:


def book_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    title = soup.h1.text
    price = soup.select_one('.price_color').text
    availability = soup.select_one('.instock.availability').text.strip()
    rating = soup.select_one('p.star-rating').get('class')[1]
    rating = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}[rating]
    description = soup.select_one('#product_description + p').text if soup.select_one('#product_description + p') else 'No description available'
    upc = soup.select_one('th:contains("UPC") + td').text

    dic = {
        "Title": title,
        "Price": price,
        "Availability": availability,
        "Rating": rating,
        "Description": description,
        "UPC": upc
    }
    return dic



# Collect and store all the information from the books in a Pandas DataFrame

Start with the following dictionary:

```python
books_dict = {"Title": [], "Price": [], "Availability": [], "Rating": [], "Description": [], "UPC": [], "Category": [] }
```

Then, iterate over all the categories and all the books in a given category to collect any book information using the previous function. Fill the previous dictionary with the information about each book.

Show the first five rows of the previous final Pandas DataFrame.

Tip: You can use the function `tqdm` from the library `tqdm` to show a progress bar if in iterable of a for loop as shown below :wink: :

```python
from tqdm import tqdm

for elem in tqdm(iterable):
    # some code
```





In [ ]:
from tqdm import tqdm

books_dict = {"Title": [], "Price": [], "Availability": [], "Rating": [], "Description": [], "UPC": [], "Category": []}

for category_url, category_name in tqdm(zip(cat_url, cat_names)):
    book_urls = get_book_urls(category_url)
    for book_url in book_urls:
        try:
            book_info = book_details(book_url)
            books_dict["Title"].append(book_info["Title"])
            books_dict["Price"].append(book_info["Price"])
            books_dict["Availability"].append(book_info["Availability"])
            books_dict["Rating"].append(book_info["Rating"])
            books_dict["Description"].append(book_info["Description"])
            books_dict["UPC"].append(book_info["UPC"])
            books_dict["Category"].append(category_name)
        except Exception as e:
            print(f"Failed to process {book_url}: {e}")

# Creating DataFrame from the dictionary
df = pd.DataFrame(books_dict)

# Display the DataFrame
print(df.head())

0it [00:00, ?it/s]

Failed to process http://books.toscrape.com/catalogue/../../a-light-in-the-attic_1000/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../tipping-the-velvet_999/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../soumission_998/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../sharp-objects_997/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../sapiens-a-brief-history-of-humankind_996/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../the-requiem-red_995/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../the-dirty-little-secrets-of-getting-your-dream-job_994/index.html: 'NoneType' object has no attribute 'text'
Fail

1it [00:05,  5.39s/it]

Failed to process http://books.toscrape.com/catalogue/../../its-only-the-himalayas_981/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../its-only-the-himalayas_981/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../see-america-a-celebration-of-our-national-parks-treasured-sites_732/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../vagabonding-an-uncommon-guide-to-the-art-of-long-term-world-travel_552/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../under-the-tuscan-sun_504/index.html: 'NoneType' object has no attribute 'text'
Failed to proces

2it [00:08,  4.06s/it]

...

Failed to process http://books.toscrape.com/catalogue/../../../delivering-the-truth-quaker-midwife-mystery-1_464/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../tipping-the-velvet_999/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../forever-and-forever-the-courtship-of-henry-longfellow-and-fanny-appleton_894/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../a-flight-of-arrows-the-pathfinders-2_876/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../the-house-by-the-lake_846/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../mrs-houdini_821/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../th

4it [00:19,  4.98s/it]

Failed to process http://books.toscrape.com/catalogue/../../../girl-in-the-blue-coat_160/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../tsubasa-world-chronicle-2-tsubasa-world-chronicle-2_949/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../this-one-summer_947/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../the-nameless-city-the-nameless-city-1_940/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../saga-volume-5-saga-collected-editions-5_923/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/

5it [00:24,  5.15s/it]

Failed to process http://books.toscrape.com/catalogue/../../../camp-midnight_886/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../the-secret-garden_413/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../the-metamorphosis_409/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../the-pilgrims-progress_353/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../the-hound-of-the-baskervilles-sherlock-holmes-5_348/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../little-women-little-women-1_331/index.html: 'NoneType' object has no attribute 'text'
Failed to process http://books.toscrape.com/catalogue/../../../gone-with-the-wind_324/index.html: 'NoneType' object has no att